# Lab: Regularisation
We import our standard libraries at this top
level.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import statsmodels.api as sm
from statsmodels.graphics.gofplots import qqplot

from statsmodels.regression.linear_model import OLS
from statsmodels.tools import add_constant
from statsmodels.stats.outliers_influence \
     import variance_inflation_factor as VIF

from statsmodels.stats.anova import anova_lm
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet, RidgeCV, LassoCV, ElasticNetCV
from sklearn.cross_decomposition import PLSRegression
from sklearn.metrics import mean_squared_error


We  will use the `Boston` housing data set.  The `Boston` dataset records  `medv`  (median house value) for $506$ neighborhoods
around Boston.  We will build a regression model to predict  `medv`  using $13$
predictors such as  `rmvar`  (average number of rooms per house),
 `age`  (proportion of owner-occupied units built prior to 1940), and  `lstat`  (percent of
households with low socioeconomic status).  We will use `statsmodels` for this
task, a `Python` package that implements several commonly used
regression methods.

In [3]:
Boston = pd.read_csv("11_Boston.csv")

The dataset contains several features related to housing in Boston:

1. crim: per capita crime rate by town
1. zn: proportion of residential land zoned for lots over 25,000 sq.ft.
1. indus: proportion of non-retail business acres per town
1. chas: Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)
1. nox: nitrogen oxides concentration (parts per 10 million)
1. rm: average number of rooms per dwelling
1. age: proportion of owner-occupied units built prior to 1940
1. dis: weighted mean of distances to five Boston employment centers
1. rad: index of accessibility to radial highways
1. tax: full-value property-tax rate per \$10,000
1. ptratio: pupil-teacher ratio by town
1. lstat: percentage of the population that is of lower status
1. medv: median value of owner-occupied homes in \$1000s (this is our target variable)

In [4]:
# Check the basic statistics
Boston.describe()

,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,lstat,medv
count,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000
mean,3.613524,11.363636,11.136779,0.069170,0.554695,6.284634,68.574901,3.795043,9.549407,408.237154,18.455534,12.653063,22.532806
std,8.601545,23.322453,6.860353,0.253994,0.115878,0.702617,28.148861,2.105710,8.707259,168.537116,2.164946,7.141062,9.197104
min,0.006320,0.000000,0.460000,0.000000,0.385000,3.561000,2.900000,1.129600,1.000000,187.000000,12.600000,1.730000,5.000000
25%,0.082045,0.000000,5.190000,0.000000,0.449000,5.885500,45.025000,2.100175,4.000000,279.000000,17.400000,6.950000,17.025000
50%,0.256510,0.000000,9.690000,0.000000,0.538000,6.208500,77.500000,3.207450,5.000000,330.000000,19.050000,11.360000,21.200000
75%,3.677083,12.500000,18.100000,0.000000,0.624000,6.623500,94.075000,5.188425,24.000000,666.000000,20.200000,16.955000,25.000000
max,88.976200,100.000000,27.740000,1.000000,0.871000,8.780000,100.000000,12.126500,24.000000,711.000000,22.000000,37.970000,50.000000


Before we fit the model, we'll split the data into training and test sets. The training set will be used to train our regression model, and the test set will be used later to evaluate its performance. We'll use an 80-20 split for the training and test sets, respectively.

In [5]:
# Define the features and the target
y = Boston[["medv"]]
X = Boston.drop("medv", axis=1)

# Split the data into training and test sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2)

Now, let's standardize the features in both the training and test sets. Remember, we'll fit the scaler on the training data and use that same scaler to transform both the training and test data. This ensures that the scaling parameters (mean and standard deviation) used for the test set are derived from the training set, preventing any potential data leakage.

In [6]:
ss = StandardScaler().set_output(transform="pandas")
ss.fit(X_train)

X_train = ss.transform(X_train)
X_test = ss.transform(X_test)

First use scikit learns `LinearRegression`. Calculate an error metric on the test data and check the intercept and coefficients.

[Linear Regression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html)

In [7]:
lr = LinearRegression()
lr.fit(X_train,y_train)
predictions = lr.predict(X_test)
lr_mse = mean_squared_error(y_test, predictions)
print(f"MSE: {lr_mse:.2f}")

MSE: 53.01


In [8]:
# Extract coefficients with feature names
coefficients_df = pd.DataFrame({
    'Variable': lr.feature_names_in_,
    'Coefficient': lr.coef_.flatten(),
})
coefficients_df.set_index('Variable', inplace=True)
coefficients_df.loc['Intercept'] = lr.intercept_
# Display the DataFrame
print(coefficients_df)

           Coefficient
Variable              
crim         -1.137341
zn            0.806080
indus        -0.174793
chas          0.209683
nox          -1.582974
rm            2.806158
age          -0.545879
dis          -2.691602
rad           2.057592
tax          -2.193861
ptratio      -1.961805
lstat        -2.838520
Intercept    22.011634


## Exercise

One potential issue with our model is multicolinearity, as we can see with the VIF reported below:

In [9]:
columns = X_train.columns.tolist()

for i in range(len(columns)):
  print(f"VIF for {columns[i]} {VIF(X_train,i): .1f}")

VIF for crim  1.7
VIF for zn  2.3
VIF for indus  4.3
VIF for chas  1.1
VIF for nox  4.3
VIF for rm  1.9
VIF for age  3.0
VIF for dis  4.0
VIF for rad  7.5
VIF for tax  9.7
VIF for ptratio  1.8
VIF for lstat  2.9


Explore the options we have seen for addressing multicolinearity and choosing features and compare their performance on the test dataset. Compare the following approaches:
* Iteratively removing features with VIF higher than 5.
* Ridge regression
* Lasso regression
* Elastic net regression
* Partial least squares regression

Make use of cross validation to set the appropriate parameter values to ahcieve the best model. For ridge, lasso and elastic net you can follow the approach before, or you can use the model version which incorporate CV:

[RidgeCV](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.RidgeCV.html)

[ElasticNetCV](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.ElasticNetCV.html)

[LassoCV](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.[]LassoCV.html)

In [10]:
# Compare to our simple linear model MSE:
print(f"MSE: {lr_mse:.2f}")

MSE: 53.01


In [26]:
ridge=RidgeCV(cv=5,alphas=np.logspace(0.1,100,10))
ridge.fit(X_train,y_train)
predict_ridge = ridge.predict(X_test)
ridge_mse = mean_squared_error(y_test, predict_ridge)
print(f"MSE: {ridge_mse:.2f}")
ridge.score(X_test,y_test)

MSE: 53.18


0.6066668113621914

In [27]:
elasticnet=ElasticNetCV(cv=5,alphas=np.logspace(0.1,100,10),l1_ratio=[.1,.5,.7,.9,.95,.99,1])
elasticnet.fit(X_train,y_train)
predictions = elasticnet.predict(X_test)
elasticnet_mse = mean_squared_error(y_test, predictions)
print(f"MSE: {elasticnet_mse:.2f}")
elasticnet.score(X_test,y_test)

MSE: 65.02


c:\my-git\DataScience-novaIMS\MachineLearning\.venv\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:1714: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.5191036979740393

In [28]:
lasso = LassoCV(cv=5,alphas=np.logspace(0.1,100,10))
lasso.fit(X_train,y_train)
predictions = lasso.predict(X_test)
lasso_mse = mean_squared_error(y_test, predictions)
print(f"MSE: {lasso_mse:.2f}")
lasso.score(X_test,y_test)

MSE: 64.76


c:\my-git\DataScience-novaIMS\MachineLearning\.venv\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:1714: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.5210214269912102

With each of the above, compare the conclusion that you would draw about the number of rooms "rm" and the access to highways "rd".

In [30]:
coefficients_df = pd.DataFrame({
    'Variable': X_train.columns,
    'Lasso Coefficient': lasso.coef_.flatten(),
    'ridge Coefficient': ridge.coef_.flatten(),
    'ElasticNet Coefficient': elasticnet.coef_.flatten(),
})
coefficients_df.set_index('Variable', inplace=True) 
# Display the DataFrame
print(coefficients_df)

          Lasso Coefficient  ridge Coefficient  ElasticNet Coefficient
Variable                                                              
crim              -0.000000          -1.125325               -0.000000
zn                 0.000000           0.784663                0.000000
indus             -0.000000          -0.209110               -0.000000
chas               0.000000           0.214871                0.000000
nox               -0.000000          -1.551282               -0.000000
rm                 2.481033           2.813305                2.475595
age               -0.000000          -0.545999               -0.000000
dis                0.000000          -2.651014                0.000000
rad               -0.000000           1.959642               -0.000000
tax               -0.414442          -2.094629               -0.444088
ptratio           -1.203631          -1.953038               -1.202928
lstat             -2.846593          -2.825963               -2.810263


In [31]:
print("Lasso selected variables:")
print(elasticnet.alpha_, elasticnet.l1_ratio_)

Lasso selected variables:
1.2589254117941673 0.99
